In [11]:
import importlib
import numpy as np
import NN
importlib.reload(NN)
from dvclive import Live

In [12]:
layers = [
    NN.DenseLayer(2, 0, NN.dummy_activation, NN.dummy_activation, True, False),
    NN.DenseLayer(4, 2, NN.sigmoid, NN.sigmoid_prime),
    NN.DenseLayer(2, 4, NN.sigmoid, NN.sigmoid_prime, False, True)
]
X = np.array([[1, 1], [1, 2]])
y = np.array([[0], [1]])


In [13]:
learning_rate = 0.01
with Live(save_dvc_exp=True, exp_message= f'Simple NN with Learning Rate {learning_rate}') as live:
    live.log_param("Learning Rate", learning_rate)
    for i in range(100):
        live.log_param('Epoch', i)
        a2 = layers[1].forward_propagation(X)
        loss = layers[2].loss(a2, y, 2)
        live.log_metric("Loss", loss)
        NN.backward_propagate(layers, X, y, learning_rate)
        live.next_step()